# Hybrid Serverless-Container Thesis - ML Model Training
## Intelligent Router: Lambda vs ECS Platform Selection

**Objective:** Train ML models to predict the optimal platform (Lambda or ECS) for different workload types based on cost and latency metrics.

**Models:**
1. Random Forest (Baseline - Interpretable)
2. XGBoost (Target: 85%+ accuracy)
3. Neural Network (Deep Learning comparison)

**Author:** Ahamed Thesis Project  
**Date:** November 2025

## 📦 Install Required Packages

In [ ]:
!pip install pandas numpy scikit-learn xgboost matplotlib seaborn plotly imbalanced-learn --quiet
print("✅ Packages installed successfully!")

## 📂 Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import json
import warnings
warnings.filterwarnings('ignore')

# Scikit-learn
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report, roc_auc_score, roc_curve
)

# XGBoost
import xgboost as xgb

# Neural Network (Keras/TensorFlow)
try:
    from tensorflow import keras
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
    from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
    print("✅ TensorFlow/Keras imported successfully")
except ImportError:
    print("⚠️  TensorFlow not available. Installing...")
    !pip install tensorflow --quiet
    from tensorflow import keras
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
    from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Set visualization style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("✅ All libraries imported successfully!")

## 📁 Upload Data Files

**Instructions:**
1. Run the preprocessing script locally: `python ml-notebooks/01_data_preprocessing.py`
2. Upload the generated file: `ml-notebooks/processed-data/ml_training_data.csv`
3. Execute the cell below to load the data

In [ ]:
from google.colab import files

print("📤 Please upload ml_training_data.csv")
uploaded = files.upload()

# Get the uploaded filename
filename = list(uploaded.keys())[0]
print(f"\n✅ File uploaded: {filename}")

## 📊 Load and Explore Data

In [ ]:
# Load data
df = pd.read_csv(filename)

print("=" * 80)
print("DATA OVERVIEW")
print("=" * 80)
print(f"\nTotal samples: {len(df):,}")
print(f"Features: {len(df.columns)}")
print(f"\nColumns: {list(df.columns)}")

# Display first few rows
print("\n" + "=" * 80)
print("SAMPLE DATA")
print("=" * 80)
display(df.head())

# Data types
print("\n" + "=" * 80)
print("DATA TYPES")
print("=" * 80)
print(df.dtypes)

# Missing values
print("\n" + "=" * 80)
print("MISSING VALUES")
print("=" * 80)
missing = df.isnull().sum()
if missing.sum() > 0:
    print(missing[missing > 0])
else:
    print("✅ No missing values!")

# Label distribution
print("\n" + "=" * 80)
print("LABEL DISTRIBUTION")
print("=" * 80)
label_counts = df['optimal_platform'].value_counts()
print(f"Lambda (1): {label_counts.get(1, 0):,} ({label_counts.get(1, 0) / len(df) * 100:.1f}%)")
print(f"ECS (0): {label_counts.get(0, 0):,} ({label_counts.get(0, 0) / len(df) * 100:.1f}%)")

## 📈 Exploratory Data Analysis (EDA)

In [ ]:
# Workload type distribution
fig = px.histogram(
    df, 
    x='workload_type', 
    color='optimal_platform',
    title='Workload Distribution by Optimal Platform',
    labels={'optimal_platform': 'Optimal Platform (1=Lambda, 0=ECS)'},
    barmode='group'
)
fig.show()

# Cost comparison
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Lambda Cost Distribution', 'ECS Cost Distribution')
)

fig.add_trace(
    go.Histogram(x=df['lambda_cost_usd'], name='Lambda Cost', nbinsx=50),
    row=1, col=1
)

fig.add_trace(
    go.Histogram(x=df['ecs_cost_usd'], name='ECS Cost', nbinsx=50),
    row=1, col=2
)

fig.update_layout(title_text="Cost Distribution by Platform", showlegend=False)
fig.show()

# Latency comparison
fig = px.box(
    df, 
    x='workload_type', 
    y='lambda_latency_ms',
    color='optimal_platform',
    title='Lambda Latency by Workload Type and Optimal Platform',
    labels={'lambda_latency_ms': 'Latency (ms)'}
)
fig.show()

## 🎯 Feature Selection and Preprocessing

In [ ]:
# Define features for ML models
# These are the features that will be available at runtime for prediction
FEATURE_COLUMNS = [
    'workload_type_encoded',  # Type of workload
    'payload_size_kb',        # Size of payload
    'hour_of_day',            # Time of day
    'day_of_week',            # Day of week
    'is_weekend',             # Weekend flag
]

TARGET_COLUMN = 'optimal_platform'  # 1 = Lambda, 0 = ECS

# Extract features and target
X = df[FEATURE_COLUMNS].copy()
y = df[TARGET_COLUMN].copy()

print("=" * 80)
print("FEATURE SELECTION")
print("=" * 80)
print(f"\nFeatures selected: {len(FEATURE_COLUMNS)}")
for i, col in enumerate(FEATURE_COLUMNS, 1):
    print(f"  {i}. {col}")

print(f"\nTarget variable: {TARGET_COLUMN}")
print(f"Samples: {len(X):,}")

# Check for any missing values in selected features
print("\n" + "=" * 80)
print("FEATURE QUALITY CHECK")
print("=" * 80)
missing_features = X.isnull().sum()
if missing_features.sum() > 0:
    print("⚠️  Missing values detected:")
    print(missing_features[missing_features > 0])
    print("\nFilling missing values with median...")
    X = X.fillna(X.median())
else:
    print("✅ No missing values in selected features")

# Display feature statistics
print("\n" + "=" * 80)
print("FEATURE STATISTICS")
print("=" * 80)
display(X.describe())

## 🔀 Train-Test Split

In [ ]:
# Split data into train (70%), validation (15%), and test (15%)
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.30, random_state=42, stratify=y
)

X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.50, random_state=42, stratify=y_temp
)

print("=" * 80)
print("TRAIN-VALIDATION-TEST SPLIT")
print("=" * 80)
print(f"\nTotal samples: {len(X):,}")
print(f"\nTraining set: {len(X_train):,} ({len(X_train)/len(X)*100:.1f}%)")
print(f"Validation set: {len(X_val):,} ({len(X_val)/len(X)*100:.1f}%)")
print(f"Test set: {len(X_test):,} ({len(X_test)/len(X)*100:.1f}%)")

print("\n" + "=" * 80)
print("LABEL DISTRIBUTION IN SPLITS")
print("=" * 80)
print("\nTraining set:")
print(f"  Lambda (1): {(y_train == 1).sum():,} ({(y_train == 1).mean()*100:.1f}%)")
print(f"  ECS (0): {(y_train == 0).sum():,} ({(y_train == 0).mean()*100:.1f}%)")

print("\nValidation set:")
print(f"  Lambda (1): {(y_val == 1).sum():,} ({(y_val == 1).mean()*100:.1f}%)")
print(f"  ECS (0): {(y_val == 0).sum():,} ({(y_val == 0).mean()*100:.1f}%)")

print("\nTest set:")
print(f"  Lambda (1): {(y_test == 1).sum():,} ({(y_test == 1).mean()*100:.1f}%)")
print(f"  ECS (0): {(y_test == 0).sum():,} ({(y_test == 0).mean()*100:.1f}%)")

## 📏 Feature Scaling

In [ ]:
# Standardize features for Neural Network (Random Forest and XGBoost don't require scaling)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

print("✅ Features scaled using StandardScaler")
print(f"   Mean: {scaler.mean_}")
print(f"   Std: {scaler.scale_}")

# 🌲 Model 1: Random Forest Classifier

**Baseline model** - Interpretable and robust to feature scaling

In [ ]:
print("=" * 80)
print("TRAINING RANDOM FOREST CLASSIFIER")
print("=" * 80)

# Train Random Forest
rf_model = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    min_samples_split=20,
    min_samples_leaf=10,
    random_state=42,
    n_jobs=-1,
    verbose=1
)

rf_model.fit(X_train, y_train)

print("\n✅ Random Forest training complete!")

# Predictions
y_train_pred_rf = rf_model.predict(X_train)
y_val_pred_rf = rf_model.predict(X_val)
y_test_pred_rf = rf_model.predict(X_test)

# Evaluation
print("\n" + "=" * 80)
print("RANDOM FOREST PERFORMANCE")
print("=" * 80)

print("\n📊 Training Set:")
print(f"  Accuracy: {accuracy_score(y_train, y_train_pred_rf):.4f}")
print(f"  Precision: {precision_score(y_train, y_train_pred_rf):.4f}")
print(f"  Recall: {recall_score(y_train, y_train_pred_rf):.4f}")
print(f"  F1-Score: {f1_score(y_train, y_train_pred_rf):.4f}")

print("\n📊 Validation Set:")
print(f"  Accuracy: {accuracy_score(y_val, y_val_pred_rf):.4f}")
print(f"  Precision: {precision_score(y_val, y_val_pred_rf):.4f}")
print(f"  Recall: {recall_score(y_val, y_val_pred_rf):.4f}")
print(f"  F1-Score: {f1_score(y_val, y_val_pred_rf):.4f}")

print("\n📊 Test Set:")
print(f"  Accuracy: {accuracy_score(y_test, y_test_pred_rf):.4f}")
print(f"  Precision: {precision_score(y_test, y_test_pred_rf):.4f}")
print(f"  Recall: {recall_score(y_test, y_test_pred_rf):.4f}")
print(f"  F1-Score: {f1_score(y_test, y_test_pred_rf):.4f}")

# Feature importance
print("\n" + "=" * 80)
print("FEATURE IMPORTANCE")
print("=" * 80)
feature_importance = pd.DataFrame({
    'feature': FEATURE_COLUMNS,
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=False)

print(feature_importance.to_string(index=False))

# Visualize feature importance
fig = px.bar(
    feature_importance,
    x='importance',
    y='feature',
    orientation='h',
    title='Random Forest Feature Importance'
)
fig.show()

# 🚀 Model 2: XGBoost Classifier

**Target: 85%+ accuracy** - Gradient boosting for high performance

In [ ]:
print("=" * 80)
print("TRAINING XGBOOST CLASSIFIER")
print("=" * 80)

# Train XGBoost
xgb_model = xgb.XGBClassifier(
    n_estimators=200,
    max_depth=6,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=-1,
    eval_metric='logloss'
)

xgb_model.fit(
    X_train, y_train,
    eval_set=[(X_train, y_train), (X_val, y_val)],
    verbose=50
)

print("\n✅ XGBoost training complete!")

# Predictions
y_train_pred_xgb = xgb_model.predict(X_train)
y_val_pred_xgb = xgb_model.predict(X_val)
y_test_pred_xgb = xgb_model.predict(X_test)

# Evaluation
print("\n" + "=" * 80)
print("XGBOOST PERFORMANCE")
print("=" * 80)

print("\n📊 Training Set:")
print(f"  Accuracy: {accuracy_score(y_train, y_train_pred_xgb):.4f}")
print(f"  Precision: {precision_score(y_train, y_train_pred_xgb):.4f}")
print(f"  Recall: {recall_score(y_train, y_train_pred_xgb):.4f}")
print(f"  F1-Score: {f1_score(y_train, y_train_pred_xgb):.4f}")

print("\n📊 Validation Set:")
print(f"  Accuracy: {accuracy_score(y_val, y_val_pred_xgb):.4f}")
print(f"  Precision: {precision_score(y_val, y_val_pred_xgb):.4f}")
print(f"  Recall: {recall_score(y_val, y_val_pred_xgb):.4f}")
print(f"  F1-Score: {f1_score(y_val, y_val_pred_xgb):.4f}")

print("\n📊 Test Set:")
print(f"  Accuracy: {accuracy_score(y_test, y_test_pred_xgb):.4f}")
print(f"  Precision: {precision_score(y_test, y_test_pred_xgb):.4f}")
print(f"  Recall: {recall_score(y_test, y_test_pred_xgb):.4f}")
print(f"  F1-Score: {f1_score(y_test, y_test_pred_xgb):.4f}")

# Feature importance
print("\n" + "=" * 80)
print("FEATURE IMPORTANCE")
print("=" * 80)
xgb_feature_importance = pd.DataFrame({
    'feature': FEATURE_COLUMNS,
    'importance': xgb_model.feature_importances_
}).sort_values('importance', ascending=False)

print(xgb_feature_importance.to_string(index=False))

# Visualize feature importance
fig = px.bar(
    xgb_feature_importance,
    x='importance',
    y='feature',
    orientation='h',
    title='XGBoost Feature Importance'
)
fig.show()

# 🧠 Model 3: Neural Network (Deep Learning)

**Comparison model** - Deep learning approach with TensorFlow/Keras

In [ ]:
print("=" * 80)
print("BUILDING NEURAL NETWORK")
print("=" * 80)

# Build Neural Network architecture
nn_model = Sequential([
    Dense(64, activation='relu', input_shape=(len(FEATURE_COLUMNS),)),
    BatchNormalization(),
    Dropout(0.3),
    
    Dense(32, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),
    
    Dense(16, activation='relu'),
    Dropout(0.2),
    
    Dense(1, activation='sigmoid')  # Binary classification
])

# Compile model
nn_model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy', 'precision', 'recall']
)

print(nn_model.summary())

# Callbacks
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True,
    verbose=1
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=5,
    min_lr=0.00001,
    verbose=1
)

print("\n" + "=" * 80)
print("TRAINING NEURAL NETWORK")
print("=" * 80)

# Train model
history = nn_model.fit(
    X_train_scaled, y_train,
    validation_data=(X_val_scaled, y_val),
    epochs=100,
    batch_size=32,
    callbacks=[early_stopping, reduce_lr],
    verbose=1
)

print("\n✅ Neural Network training complete!")

# Predictions
y_train_pred_nn_prob = nn_model.predict(X_train_scaled)
y_val_pred_nn_prob = nn_model.predict(X_val_scaled)
y_test_pred_nn_prob = nn_model.predict(X_test_scaled)

y_train_pred_nn = (y_train_pred_nn_prob > 0.5).astype(int).flatten()
y_val_pred_nn = (y_val_pred_nn_prob > 0.5).astype(int).flatten()
y_test_pred_nn = (y_test_pred_nn_prob > 0.5).astype(int).flatten()

# Evaluation
print("\n" + "=" * 80)
print("NEURAL NETWORK PERFORMANCE")
print("=" * 80)

print("\n📊 Training Set:")
print(f"  Accuracy: {accuracy_score(y_train, y_train_pred_nn):.4f}")
print(f"  Precision: {precision_score(y_train, y_train_pred_nn):.4f}")
print(f"  Recall: {recall_score(y_train, y_train_pred_nn):.4f}")
print(f"  F1-Score: {f1_score(y_train, y_train_pred_nn):.4f}")

print("\n📊 Validation Set:")
print(f"  Accuracy: {accuracy_score(y_val, y_val_pred_nn):.4f}")
print(f"  Precision: {precision_score(y_val, y_val_pred_nn):.4f}")
print(f"  Recall: {recall_score(y_val, y_val_pred_nn):.4f}")
print(f"  F1-Score: {f1_score(y_val, y_val_pred_nn):.4f}")

print("\n📊 Test Set:")
print(f"  Accuracy: {accuracy_score(y_test, y_test_pred_nn):.4f}")
print(f"  Precision: {precision_score(y_test, y_test_pred_nn):.4f}")
print(f"  Recall: {recall_score(y_test, y_test_pred_nn):.4f}")
print(f"  F1-Score: {f1_score(y_test, y_test_pred_nn):.4f}")

# Plot training history
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Model Accuracy', 'Model Loss')
)

fig.add_trace(
    go.Scatter(y=history.history['accuracy'], name='Train Accuracy', mode='lines'),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(y=history.history['val_accuracy'], name='Val Accuracy', mode='lines'),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(y=history.history['loss'], name='Train Loss', mode='lines'),
    row=1, col=2
)
fig.add_trace(
    go.Scatter(y=history.history['val_loss'], name='Val Loss', mode='lines'),
    row=1, col=2
)

fig.update_xaxes(title_text="Epoch", row=1, col=1)
fig.update_xaxes(title_text="Epoch", row=1, col=2)
fig.update_yaxes(title_text="Accuracy", row=1, col=1)
fig.update_yaxes(title_text="Loss", row=1, col=2)

fig.update_layout(title_text="Neural Network Training History")
fig.show()

# 📊 Model Comparison and Evaluation

In [ ]:
# Compile results
results = pd.DataFrame({
    'Model': ['Random Forest', 'XGBoost', 'Neural Network'],
    'Train_Accuracy': [
        accuracy_score(y_train, y_train_pred_rf),
        accuracy_score(y_train, y_train_pred_xgb),
        accuracy_score(y_train, y_train_pred_nn)
    ],
    'Val_Accuracy': [
        accuracy_score(y_val, y_val_pred_rf),
        accuracy_score(y_val, y_val_pred_xgb),
        accuracy_score(y_val, y_val_pred_nn)
    ],
    'Test_Accuracy': [
        accuracy_score(y_test, y_test_pred_rf),
        accuracy_score(y_test, y_test_pred_xgb),
        accuracy_score(y_test, y_test_pred_nn)
    ],
    'Test_Precision': [
        precision_score(y_test, y_test_pred_rf),
        precision_score(y_test, y_test_pred_xgb),
        precision_score(y_test, y_test_pred_nn)
    ],
    'Test_Recall': [
        recall_score(y_test, y_test_pred_rf),
        recall_score(y_test, y_test_pred_xgb),
        recall_score(y_test, y_test_pred_nn)
    ],
    'Test_F1': [
        f1_score(y_test, y_test_pred_rf),
        f1_score(y_test, y_test_pred_xgb),
        f1_score(y_test, y_test_pred_nn)
    ]
})

print("=" * 80)
print("MODEL COMPARISON SUMMARY")
print("=" * 80)
display(results)

# Find best model
best_model_idx = results['Test_Accuracy'].idxmax()
best_model_name = results.loc[best_model_idx, 'Model']
best_accuracy = results.loc[best_model_idx, 'Test_Accuracy']

print(f"\n🏆 Best Model: {best_model_name} (Test Accuracy: {best_accuracy:.4f})")

if best_accuracy >= 0.85:
    print("✅ Target accuracy of 85% achieved!")
else:
    print(f"⚠️  Target accuracy of 85% not yet achieved. Current: {best_accuracy:.4f}")

# Visualize comparison
fig = go.Figure()

metrics = ['Train_Accuracy', 'Val_Accuracy', 'Test_Accuracy', 'Test_Precision', 'Test_Recall', 'Test_F1']
for model in results['Model']:
    model_data = results[results['Model'] == model]
    fig.add_trace(go.Bar(
        name=model,
        x=metrics,
        y=model_data[metrics].values[0]
    ))

fig.update_layout(
    title='Model Performance Comparison',
    xaxis_title='Metrics',
    yaxis_title='Score',
    barmode='group'
)
fig.show()

# 📉 Confusion Matrices

In [ ]:
# Create confusion matrices for all models
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

models_cm = [
    ('Random Forest', y_test_pred_rf),
    ('XGBoost', y_test_pred_xgb),
    ('Neural Network', y_test_pred_nn)
]

for idx, (model_name, y_pred) in enumerate(models_cm):
    cm = confusion_matrix(y_test, y_pred)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[idx])
    axes[idx].set_title(f'{model_name}\nConfusion Matrix')
    axes[idx].set_xlabel('Predicted')
    axes[idx].set_ylabel('Actual')
    axes[idx].set_xticklabels(['ECS (0)', 'Lambda (1)'])
    axes[idx].set_yticklabels(['ECS (0)', 'Lambda (1)'])

plt.tight_layout()
plt.show()

# Print classification reports
print("\n" + "=" * 80)
print("DETAILED CLASSIFICATION REPORTS")
print("=" * 80)

for model_name, y_pred in models_cm:
    print(f"\n{model_name}:")
    print("-" * 80)
    print(classification_report(y_test, y_pred, target_names=['ECS (0)', 'Lambda (1)']))

# 💾 Save Best Model

In [ ]:
import pickle

# Save the best performing model
if best_model_name == 'Random Forest':
    best_model = rf_model
elif best_model_name == 'XGBoost':
    best_model = xgb_model
else:
    best_model = nn_model

# Save model
model_filename = f'best_model_{best_model_name.replace(" ", "_").lower()}.pkl'

if best_model_name != 'Neural Network':
    with open(model_filename, 'wb') as f:
        pickle.dump(best_model, f)
    print(f"✅ Model saved: {model_filename}")
else:
    nn_model.save('best_model_neural_network.h5')
    print(f"✅ Neural Network saved: best_model_neural_network.h5")

# Save scaler (needed for Neural Network)
with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)
print(f"✅ Scaler saved: scaler.pkl")

# Save feature names
with open('feature_columns.json', 'w') as f:
    json.dump(FEATURE_COLUMNS, f)
print(f"✅ Feature columns saved: feature_columns.json")

# Save model metadata
metadata = {
    'best_model': best_model_name,
    'test_accuracy': float(best_accuracy),
    'feature_columns': FEATURE_COLUMNS,
    'training_samples': len(X_train),
    'test_samples': len(X_test),
    'model_performance': results.to_dict('records')
}

with open('model_metadata.json', 'w') as f:
    json.dump(metadata, f, indent=2)
print(f"✅ Metadata saved: model_metadata.json")

print("\n" + "=" * 80)
print("🎉 MODEL TRAINING COMPLETE!")
print("=" * 80)
print(f"\nBest Model: {best_model_name}")
print(f"Test Accuracy: {best_accuracy:.4f}")
print(f"\nFiles ready for download and AWS deployment:")
print(f"  1. {model_filename if best_model_name != 'Neural Network' else 'best_model_neural_network.h5'}")
print(f"  2. scaler.pkl")
print(f"  3. feature_columns.json")
print(f"  4. model_metadata.json")

# 📥 Download Model Files

In [ ]:
from google.colab import files

# Download all model files
print("📥 Downloading model files...")

if best_model_name != 'Neural Network':
    files.download(model_filename)
else:
    files.download('best_model_neural_network.h5')

files.download('scaler.pkl')
files.download('feature_columns.json')
files.download('model_metadata.json')

print("\n✅ All files downloaded successfully!")
print("\n🚀 Next Steps:")
print("  1. Upload model files to your repository")
print("  2. Create intelligent router API on AWS Lambda")
print("  3. Deploy and test the hybrid routing system")
print("  4. Evaluate against Lambda-only and ECS-only baselines")